# TM10007 Group Assignment Machine Learning
#### Sara Arman, Judith Essenburg, George Franssen, Naomi Verkerk

## Google colab environment

In [1]:
# Run this to use from colab environment
#!pip install -q --upgrade git+https://github.com/naomiverkerk/TM10007.git

## Import


In [2]:
from load_data import load_data
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

## Loading Data

In [6]:
data = load_data() 

print(f'The number of samples/patients: {len(data.index)}') #rows
print(f'The number of columns/features: {len(data.columns)}') #columns

data2 = data.iloc[155:158, 7:9]
data2



The number of samples/patients: 167
The number of columns/features: 725


,VOLUME_ET_OVER_ED,VOLUME_NET_OVER_ED
ID,,
TCGA-HT-7602,0,2.7936340407
TCGA-HT-7680,0.017069507,1.315581046
TCGA-HT-7686,#DIV/0!,#DIV/0!


In [7]:
data = data.replace('#DIV/0!', np.nan)
data2 = data.iloc[155:158, 7:9]
data2

,VOLUME_ET_OVER_ED,VOLUME_NET_OVER_ED
ID,,
TCGA-HT-7602,0,2.7936340407
TCGA-HT-7680,0.017069507,1.315581046
TCGA-HT-7686,NaN,NaN


In [ ]:
y = data.label
ax = sns.countplot(y,label="Count")     
GBM, LGG = y.value_counts()
print('Number of GBM: ',GBM)
print('Number of LGG: ',LGG)

## Splitting in train and test data

In [ ]:
# the code to split, after that we inspect the data

## Checking/exploring the dataset

In [70]:
data.dtypes == 'object'
# Seperate numerical variables and categorical variables
num_vars = data.columns[data.dtypes != 'object']
cat_vars = data.columns[data.dtypes == 'object']

print(len(num_vars))
print(len(cat_vars))
print(cat_vars)



722
3
Index(['VOLUME_ET_OVER_ED', 'VOLUME_NET_OVER_ED', 'label'], dtype='object')


In [74]:
data.columns

Index(['VOLUME_ET', 'VOLUME_NET', 'VOLUME_ED', 'VOLUME_TC', 'VOLUME_WT',
       'VOLUME_BRAIN', 'VOLUME_ET_OVER_NET', 'VOLUME_ET_OVER_ED',
       'VOLUME_NET_OVER_ED', 'VOLUME_ET_over_TC',
       ...
       'TGM_T_4', 'TGM_Cog_X_5', 'TGM_Cog_Y_5', 'TGM_Cog_Z_5', 'TGM_T_5',
       'TGM_Cog_X_6', 'TGM_Cog_Y_6', 'TGM_Cog_Z_6', 'TGM_T_6', 'label'],
      dtype='object', length=725)

In [ ]:
GBM = data[data.label == 'GBM']
LGG = data[data.label == 'LGG']

In [ ]:
# # to check which features have most missing values
# missing_values = data[num_vars].isnull().sum().sort_values(ascending=False)
# missing_values_multiple = (missing_values > 0).sum()

# print(missing_values)
# print(f'The total number of features with 1 or more missing values is {missing_values_multiple}')



In [103]:
# saving missing values in a variable
a = pd.isna(data).sum()/len(data)*100

# saving column names in a variable
variables = data.columns

# new variable to store variables having missing values less than a threshold

variable = [ ]

for i in range(data.columns.shape[0]):
    if a[i]<=40: #setting the threshold as 40%
        variable.append(variables[i])

variable   

# creating a new dataframe using the above variables
new_data = data[variable]

# percentage of missing values in each variable of new data
p = pd.isna(new_data).sum()/len(new_data)*100


In [104]:
# Finding data point with inf
x = data.iloc[:,:].isin([np.inf, -np.inf]).sum()
x[x>0]

DIST_Vent_ED    1
dtype: int64

In [105]:
data_columns_name = new_data.columns

In [110]:
total_upper = []
total_lower = []
len_out = 167

for i in range(0,len_out):
    Q1 = np.percentile(new_data[data_columns_name[i]], 25, interpolation = 'midpoint')
    Q3 = np.percentile(new_data[data_columns_name[i]], 75, interpolation = 'midpoint')

    IQR = Q3 - Q1

    upper = Q3 + 1.5*IQR
    lower = Q1 - 1.5*IQR

    upper_list = new_data[data_columns_name[i]] >= (Q3+1.5*IQR)
        
    # Below Lower bound
    lower_list = new_data[data_columns_name[i]] <= (Q1-1.5*IQR)
    
    total_upper.append(upper_list.sum())
    total_lower.append(lower_list.sum())

    upper_list_values = new_data[data_columns_name[i]][upper_list]
    lower_list_values = new_data[data_columns_name[i]][lower_list]



In [111]:
Id = list(new_data.index)

df = {  'ID': Id[0:len_out],
        'Upper outliers':  total_upper,
        'Lower outliers': total_lower,

        }

df = pd.DataFrame(df)
df['Total'] = df['Upper outliers'] + df['Lower outliers']

df = pd.DataFrame(df)
df


,ID,Upper outliers,Lower outliers,Total
0,TCGA-02-0006,7,0,7
1,TCGA-02-0009,9,0,9
2,TCGA-02-0011,3,0,3
3,TCGA-02-0027,4,0,4
4,TCGA-02-0033,1,0,1
...,...,...,...,...
162,TCGA-HT-8018,3,0,3
163,TCGA-HT-8111,3,0,3
164,TCGA-HT-8114,0,0,0
165,TCGA-HT-8563,5,0,5
